In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_pickle('Cleaned_ed.pkl')

# MODELING- Random Forest

In [3]:
from sklearn.model_selection import train_test_split


/Users/leticiabeeck/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
binary = {'Meeting Target':1,'Exceeding Target':1,
           'Not Meeting Target':0,'Approaching Target':0}
df['Student Achievement Rating'] = df['Student Achievement Rating'].map(binary)

In [5]:
df['Student Achievement Rating'].value_counts()

1    589
0    206
Name: Student Achievement Rating, dtype: int64

## Train Test Split 

In [6]:
df['City'] = [value if value in df['City'].value_counts().index[:4] else 'Other' for value in df['City']]

In [7]:
cats= ['City',
       'Rigorous Instruction Rating', 'Collaborative Teachers Rating',
       'Supportive Environment Rating', 'Effective School Leadership Rating',
       'Strong Family-Community Ties Rating', 'Trust Rating']
df.drop(['School Name', 'Location Code', 'Address (Full)'], axis=1, inplace=True)

In [8]:
X = df.drop('Student Achievement Rating',axis=1)
y = df['Student Achievement Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


X_train = pd.get_dummies(X_train, columns=cats, drop_first=True)
X_test = pd.get_dummies(X_test, columns=cats, drop_first=True)

In [9]:
[col for col in X_train.columns if col not in X_test.columns]

['Collaborative Teachers Rating_Not Meeting Target']

In [10]:
edu_cats = {
    "PK,0K,01,02,03,04,05"     : "has_elementary",
    "PK,0K,01,02,03,04,05,06,07,08" : "has_middle_school",
    "0K,01,02,03,04,05,06,07,08,09,10,11,12"   : "has_high_school",
    '0K,01,02,03,04,05': "has_elementary",
    '0K,01,02,03,04,05,06,07,08':"has_middle_school",
    '04,05,06,07,08':'has_middle_school',
    '03,04,05':'has_elementary',
    '0K,01,02,03,04,05,06,07,08,SE':'has_middle_school', 
    'PK,0K,01,02,03,04,05,06':'has_elementary',
    'PK,0K,01,02,03,04,05,SE':'has_elementary',
    'PK,0K,01,02,03':'has_elementary', 
    '04,05':'has_elementary',
    '0K,01,02,03,04':'has_elementary', 
    '0K,01,02,03,04,05,06':'has_elementary', 
    'PK,0K,01,02':'has_elementary',
    'PK,0K,01,02,03,04':'has_elementary', 
    '02,03,04,05':'has_elementary',
    'PK,0K,01,02,03,04,05,07,08':'has_middle_school',
    'PK,0K,01,02,03,04,05,06,07,08,SE':'has_middle_school', 
    '06,07,08':'has_middle_school',
    '0K':'pre_school',
    '03,04,05,06':'has_elementary',
    '0K,01,02,03,04,05,SE':'has_elementary', 
    '01,05,06,07,08,K':'has_middle_school',
    '01,02,03,04,05,06,07,08':'has_middle_school',
    '0K,01,02,03,04,05,06,07':'has_middle_school'
}

X_train['Grade'] = df['Grades'].map(edu_cats)
X_test['Grade']=df['Grades'].map(edu_cats)

In [11]:
X_train=pd.get_dummies(X_train, columns=['Grade'], drop_first=True)
X_test=pd.get_dummies(X_test,columns=['Grade'],drop_first=True)

In [12]:
X_train.drop('Grades',axis=1,inplace=True)
X_test.drop('Grades',axis=1,inplace=True)

# Grid-Searching

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [14]:
rf   = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

param_grid = {
    'n_estimators':[5,8,10,12],
    'max_depth':[25,30,27]
    
    
    
}

grid = GridSearchCV(estimator=rf, param_grid=param_grid,cv=3)

grid.fit(X_train,y_train)



GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 8, 10, 12], 'max_depth': [25, 30, 27]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=27, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [16]:
grid.score(X_train,y_train)

0.9811320754716981

In [29]:
grid.fit(X_test,y_test)
grid.score(X_test,y_test)

0.9874213836477987

In [20]:
[col for col in X_train.columns if col not in X_test.columns]

['Collaborative Teachers Rating_Not Meeting Target', 'Grade_pre_school']

In [21]:
X_train.drop(['Collaborative Teachers Rating_Not Meeting Target','Grade_pre_school'],axis=1,inplace=True)